In [1]:
import os

from azure.ai.agents.models import FilePurpose,VectorStore,FileSearchTool
from azure.ai.projects.aio import AIProjectClient
from azure.identity.aio import AzureCliCredential
from dotenv import load_dotenv

from agent_framework import AgentRunResponse,ChatAgent,HostedFileSearchTool,HostedVectorStoreContent
from agent_framework.azure import AzureAIAgentClient

In [2]:
load_dotenv()

True

In [3]:
async def create_vector_store(client: AzureAIAgentClient) -> tuple[str, VectorStore]:
    """Create a vector store with sample documents."""
    file_path = '../files/demo.md'
    file = await client.project_client.agents.files.upload_and_poll(file_path=file_path, purpose=FilePurpose.AGENTS)
    print(f"Uploaded file, file ID: {file.id}")


    vector_store = await client.project_client.agents.vector_stores.create_and_poll(file_ids=[file.id], name="graph_knowledge_base")

    print(f"Created vector store, ID: {vector_store.id}")


    return file.id, vector_store

In [4]:
async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential) as chat_client,
    ):
        file_id, vector_store = await create_vector_store(chat_client)

        file_search = FileSearchTool(vector_store_ids=[vector_store.id])
        
        agent = chat_client.create_agent(
            name="PythonRAGDemo",
            instructions="""
                You are a helpful assistant that helps people find information in a set of files.  If you can't find the answer in the files, just say you don't know. Do not make up an answer.
                """,
            tools=file_search.definitions,  # Tools available to the agent
            tool_resources=file_search.resources,  # Resources for the tool
        )
                

        print("Agent created. You can now ask questions about the uploaded document.")

        query = "What is GraphRAG?"
        response = await AgentRunResponse.from_agent_response_generator(agent.run_stream(query, tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}}))
        print(f"Assistant: {response}")

Uploaded file, file ID: assistant-CiUQ5xjyFBC7TJyJ331FCF
Created vector store, ID: vs_BEbCqzLGHRRQsP0mMALlV5CV
Agent created. You can now ask questions about the uploaded document.
Assistant: GraphRAG is an AI-based content interpretation and search system that uses large language models to create a knowledge graph from a user-provided dataset. It connects information across large volumes of data to answer complex, thematic, or multi-document questions that are difficult to address through traditional keyword or vector search methods. The system is designed to support critical analysis and discovery, especially in contexts where information is noisy or spread across many sources. It emphasizes transparency, grounded responses, and resilience to injection attacks, although it relies on well-constructed indexing and human oversight for optimal performance【4:0†demo.md】.
